# Get Masked Betas for each participant

First, imports and paths:

In [ ]:
# Imports and setup paths
import nilearn
from os.path import join
from os import listdir
import numpy as np

steph_dir = '/imaging/astle/users/su01/RED'
perm_dir = join(steph_dir, '3 Permutations_SnPM/Outputs/Go/1 Sad vs Happy')
group_mask_file= join(perm_dir, 'SadvsHappy_neg_SnPM_filtered.nii')

read in the .nii file from the group level as data

Reading in the data gives us a numpy array, which is much more flexible than niilearns own formats

This is a 3D array representing voxels, nan values represent no significance, anything not a nan is a significant beta


In [ ]:
group_mask = nilearn.image.get_data(group_mask_file)


 we now convert this image of NaN's and numbers into a boolean array, which we can later use as an index
 Numpy allows boolean indexing, so we can just use our mask as an array to blank out other files
 See: https://numpy.org/devdocs/reference/arrays.indexing.html#boolean-array-indexing

 we use numpy is nan, which returns True if a number is NaN -- we want to reverse this, so we put the '~' preceding


In [ ]:
group_boolean_mask = ~np.isnan(group_mask) # get a mask of true and false for if that voxel was significant

now we want to loop through the lower level files and get a beta value for each participant using this mask

First, set up paths to the 1st level directories and contrast names.

Then make a list of all the subject folders

In [ ]:
first_level_directory = join(steph_dir, '1stlevel_output_go', '1stLevel')
cont_name = 'con_0004.nii' # make sure this matches the GLM contrast ie. 'Happy Go > Sad Go'
sublist = listdir(first_level_directory) # list all directories in this folder

Now loop through the subject folders using enumerate

We then mask these lower level statsitics and can use them to get a summary stat for each child

In [ ]:
beta_vals = [] # our beta value list holder
for i, sub_dir in enumerate(sublist):
    this_beta = nilearn.image.get_data(join(first_level_directory, sub_dir, cont_name))# read in the betas from this participant, again as a numpy array
    this_masked_vals = this_beta[group_boolean_mask] # mask them using the boolean mask
    this_masked_vals = np.abs(this_masked_vals) # take absolute values (for ease)
    this_beta_stat = np.mean(this_masked_vals) # take sum (but you can use average - np.mean(), peak np.max() or whatever function you'd like)
    beta_vals.append((sub_dir, this_beta_stat)) # append this participants value along with their ID


now we have the summed/averaged (depending) activity for each participant, masked by the group level contrast

If significant, we can also go back and get the masked values only and look at which voxels most predict the covariates
but for now these are pretty good as is

In [ ]:
print(beta_vals)

## Final analysis steps
Can look at any of the covariates

May do this on local computer so will need to change some directories and paths if that is the case

In [ ]:
#Read into pandas
b_df = pd.DataFrame(np.array(beta_vals), columns=['sub_id', 'summed_betas'])
b_df.summed_betas = b_df.summed_betas.astype('float')
# load BIDS2RED and merge to get RED ids for data
bid2red = pd.read_excel('/imaging/astle/users/ai05/RED/RED_fmri_finalids.xlsx')
main_df = pd.merge(b_df, bid2red, on='sub_id')

#load in all metadata
META_PATH = join('/imaging', 'astle','users','ai05', 'RED', 'RED_MEG', 'worms', 'Combined3.csv')
meta_df = pd.read_csv(META_PATH)
meta_df.rename(columns={'Alex_ID': 'red_id'},inplace=True)
main_df['red_id'] = main_df['red_id'].astype(str)
with_data = pd.merge(main_df, meta_df, on='red_id')


#%% Now do some analysis -- this can be whatever covariate you want really
plt.close('all')
sns.regplot(y='summed_betas', x='WASI_Mat', data=with_data)
plt.show()